# Visualizing Geospatial Polygon Data with Python : Land Surface

This notenook shows how to use several open source tools and techniques for visualizing polygon data from a shapefile on differents types of maps.

For this, we will use :
- [GeoPandas](https://geopandas.org/en/stable/) to store data like [Pandas](https://pandas.pydata.org/) but with spatial information and operations
- [Cartopy](https://scitools.org.uk/cartopy/docs/latest/index.html) to add satellites images

## Database initialisation and simplification

The land surface shapefile used here is from the Theia database CES Occupation des sols. It's automatically produced from Sentinel-2A and Sentinel-2B data, with a resolution of 10m and 24 types of land.

The link to the dataset: https://theia.cnes.fr/atdistrib/rocket/#/search?collection=OSO

The link of the S2 catalogue used at the end: https://catalogue.theia-land.fr/

In [ ]:
import numpy as np                       # linear algebra
import matplotlib.pyplot as plt          # plotting library
import pandas as pd                      # data processing
import geopandas as gpd                  # data processing

plt.ion();

In [ ]:
# Load haute-garonne shapefile with geopandas
hg = gpd.read_file("data/departement_31.shp")

# Display the first 3 values
hg.head(3)

In [ ]:
# Sum all agriculutural/countryside columns in one
countryside_columns = ['Colza', 'CerealPail','Proteagine', 'Soja', 'Tournesol', 'Mais', 'Riz', 'TuberRacin',
        'Vergers', 'Vignes',  'Serres', 'Prairie']
hg['Agriculture'] =  hg[countryside_columns].sum(axis=1)

# Same for forests and urbans columns
hg['Foret'] =  hg[['Feuillus', 'Coniferes']].sum(axis=1)
hg['Urbain'] =  hg[['UrbainDens', 'UrbainDiff', 'ZoneIndCom', 'Routes',  'Pelouse']].sum(axis=1)   

# Delete all sub columns already used
types_list = ['Foret','Urbain', "Agriculture", 'Landes',  'PlageDune', 'GlaceNeige', 'Eau']
hg_main_types = hg[ types_list + ["geometry"]]

# Display the first 5 values
hg_main_types.head()

## Plot examples

### Plot density of urban land for each polygon

With geopandas, the plot present by default spacialy information.

In [ ]:
hg_main_types.plot(column="Urbain", cmap="OrRd", legend=True);

### Plot main type of each polygon

In [ ]:
# The main type of each polygon is the column with the greatest value
hg_main_types["Type"] = hg_main_types[types_list].idxmax(axis=1)

# Plot that new column
hg_main_types.plot(column = "Type", legend=True);

### Plot urban and countryside land on a satellite image

To do this, we use Cartopy that manage projections.

In [ ]:
from osgeo import gdal, osr
import rasterio

image_crop = 'data/T31TCJ_20240708T105031_B04_crop.jp2'
crop = [ 350000,  4850020, 399800, 4800220]


"""# We previously took a S2 image and crop it to be lighter to work with
image = 'data/S2A_MSIL1C_20240708T105031_N0510_R051_T31TCJ_20240708T125024.SAFE/GRANULE/L1C_T31TCJ_A047238_20240708T105028/IMG_DATA/T31TCJ_20240708T105031_B04.jp2'
gdal.Translate(image_crop, image, projWin = crop)"""



In [ ]:
dataset = rasterio.open(image_crop)

# Get image information
epsg = dataset.crs
img_proj = [dataset.bounds[0], dataset.bounds[2], dataset.bounds[1], dataset.bounds[3]] 

# Add a threshold to the image to be visually comprehensive
img = plt.imread(image_crop)
img = np.array(img)
img[img > 10000] = 10000

# Projection on the land surface polygon into the image epsg 
hg_main_types = hg_main_types.to_crs(epsg)

# Select only urban and countryside lands 
hg_agri = hg_main_types.loc[hg_main_types["Type"] == "Agriculture"]
hg_urb = hg_main_types.loc[hg_main_types["Type"] == "Urbain"]

In [ ]:
import cartopy.crs as ccrs

# Creation of the plot   
fig = plt.figure(figsize=(8, 12))

# Make the map
ax = plt.axes(projection=ccrs.UTM(zone=30))
ax.set_title('T31TCJ S2 tiles with urban (orange) and countryside (green) area')
ax.set_extent(img_proj, crs=ccrs.UTM(zone=30))

# Add the image
ax.imshow(img, origin='upper', extent=img_proj, transform=ccrs.UTM(zone=30), cmap="gray", alpha=1.0)

# Add urban and countryside lands
ax.add_geometries(hg_agri["geometry"].values, crs=ccrs.UTM(zone=30), facecolor='green', alpha=0.7)
ax.add_geometries(hg_urb["geometry"].values, crs=ccrs.UTM(zone=30), facecolor='orange', alpha=0.7)

# Add coordinates legends
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

plt.show()